In [ ]:
%pwd

In [ ]:
import sys
sys.path.append('../param_search')
import param_search as ps

In [3]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT
nvidia-smi

data_root=../data/{data_name}
data_root=$(realpath $data_root)

case_name=Case{case_id:d}Pack
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment
output_dir=$case_dir/CorrField

mkdir -p $output_dir

fixed_name=case{case_id:d}_T{fixed_phase:02d}
fixed_image=$image_dir/$fixed_name.nii.gz
fixed_mask=$mask_dir/$fixed_name/lung_combined_mask.nii.gz

moving_name=case{case_id:d}_T{moving_phase:02d}
moving_image=$image_dir/$moving_name.nii.gz

reg_name=case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}
output_path=$output_dir/$reg_name

cd ../Lung250M-4B/corrfield
python corrfield.py -F $fixed_image -M $moving_imgae -m $fixed_mask -O $output_path

echo Done
'''
name_format = 'register_{data_name}_case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}'

In [6]:
param_space = ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    fixed_phase=[50],
    moving_phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
)

for p in param_space:
    print(name_format.format(**p))
    
print(len(param_space))

register_Emory-4DCT_case1_T00_T50
register_Emory-4DCT_case1_T10_T50
register_Emory-4DCT_case1_T20_T50
register_Emory-4DCT_case1_T30_T50
register_Emory-4DCT_case1_T40_T50
register_Emory-4DCT_case1_T50_T50
register_Emory-4DCT_case1_T60_T50
register_Emory-4DCT_case1_T70_T50
register_Emory-4DCT_case1_T80_T50
register_Emory-4DCT_case1_T90_T50
register_Emory-4DCT_case2_T00_T50
register_Emory-4DCT_case2_T10_T50
register_Emory-4DCT_case2_T20_T50
register_Emory-4DCT_case2_T30_T50
register_Emory-4DCT_case2_T40_T50
register_Emory-4DCT_case2_T50_T50
register_Emory-4DCT_case2_T60_T50
register_Emory-4DCT_case2_T70_T50
register_Emory-4DCT_case2_T80_T50
register_Emory-4DCT_case2_T90_T50
register_Emory-4DCT_case3_T00_T50
register_Emory-4DCT_case3_T10_T50
register_Emory-4DCT_case3_T20_T50
register_Emory-4DCT_case3_T30_T50
register_Emory-4DCT_case3_T40_T50
register_Emory-4DCT_case3_T50_T50
register_Emory-4DCT_case3_T60_T50
register_Emory-4DCT_case3_T70_T50
register_Emory-4DCT_case3_T80_T50
register_Emory

In [7]:
jobs = ps.submit(template, name_format, param_space, work_dir='2023-11-30_register')
jobs

100%|██████████| 100/100 [00:00<00:00, 106.03it/s]


,data_name,case_id,fixed_phase,moving_phase,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,20779811,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,20779812,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,20779813,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,20779814,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,20779815,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,20779906,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,20779907,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,20779908,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
98,Emory-4DCT,10,50,80,register_Emory-4DCT_case10_T80_T50,20779909,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN


In [8]:
ps.status(jobs)

,index,data_name,case_id,fixed_phase,moving_phase,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,,
20779811,0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779812,1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779813,2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779814,3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779815,4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20779906,95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779907,96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
20779908,97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
